In [1]:
import sys
sys.path.append('../')
sys.path.append('.')
import recoengi
import recoengi.cf as cf

import pandas as pd
from scipy import sparse
import numpy as np
import logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s %(asctime)s %(message)s")

In [2]:
dtf_ratings = pd.read_csv("sampledata/ml-latest-small/ratings.csv", usecols=["userId", "movieId", "rating"])
dtf_tmp = pd.DataFrame({"movieId": dtf_ratings.movieId.unique()})
dtf_tmp = dtf_tmp.reset_index(drop = False)
dtf_tmp = dtf_tmp.rename({"index": "movieId_new"}, axis = 1)
dtf_ratings = pd.merge(dtf_ratings, dtf_tmp, on = ["movieId"], how = "left").drop(["movieId"], axis=1).rename({"movieId_new": "movieId"}, axis=1)
dtf_ratings.userId = dtf_ratings.userId-1
dtf_ratings.rating = (dtf_ratings.rating >= 3) + 0.0
tmp_bln_split = np.random.choice([True, False], size=dtf_ratings.shape[0], replace=True, p=[0.8, 0.2])
dtf_train = dtf_ratings.loc[tmp_bln_split, ["userId", "movieId", "rating"]]
dtf_test = dtf_ratings.loc[~tmp_bln_split, ["userId", "movieId", "rating"]]

In [3]:
M = sparse.csr_matrix((dtf_train.rating, (dtf_train.userId, dtf_train.movieId)))

In [4]:
cfm = cf.CFM(M)
cfm.computeEverything(bln_bin=False, bln_norm=True, flt_ths=0.01, ntop=64, flt_lb=-1)

INFO 2019-10-13 11:00:50,414 M matrix has shape 610x9724.
INFO 2019-10-13 11:00:50,416 B matrix has shape 610x9724.
INFO 2019-10-13 11:00:50,436 B matrix has sparsity 1.1037925430403734%.
INFO 2019-10-13 11:00:50,438 Computing the similarity matrix S ...
INFO 2019-10-13 11:00:50,475 S matrix has shape 610x610.
INFO 2019-10-13 11:00:50,476 Computing the matrix SNORMALIZED ...
INFO 2019-10-13 11:00:50,480 Computing the matrix SCORES ...
INFO 2019-10-13 11:00:50,533 SCORES matrix has shape 610x9724.
INFO 2019-10-13 11:00:50,534 Computing the matrix AMOUNTS ...
INFO 2019-10-13 11:00:50,579 AMOUNTS matrix has shape 610x9724.
INFO 2019-10-13 11:00:50,579 Computing the performances ...
INFO 2019-10-13 11:00:50,750 Average global scores difference: 0.022671029948770967.
INFO 2019-10-13 11:00:50,751 Average positive scores difference: 0.7703604360768836.
WARNING 2019-10-13 11:00:50,752 The scores model could not performing better than a random model!
INFO 2019-10-13 11:00:50,924 Average global 

In [5]:
dtf_pred = pd.DataFrame(pd.Series(dict(cfm.SCORES.todok().items()))).reset_index(drop=False)
dtf_pred.columns = ["userId", "movieId", "predicted_score"]

In [6]:
dtf_test = pd.merge(dtf_test, dtf_pred, on=["userId", "movieId"], how="inner")

In [7]:
dtf_test.head()

,userId,movieId,rating,predicted_score
0,0,2,1.0,0.386412
1,0,7,1.0,0.587905
2,0,15,1.0,0.782951
3,0,16,1.0,0.700878
4,0,17,1.0,0.382538


In [8]:
abs(dtf_test.rating - dtf_test.predicted_score).mean()

0.6649715028394231